In [1]:
import numpy as np
import pandas as pd

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display

from plotly import graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import tools

init_notebook_mode(connected=True)

In [2]:
df = pd.read_csv('Airplane_Crashes_and_Fatalities_Since_1908.csv')

In [3]:
df['Date'] = df.Date.apply(lambda x: pd.to_datetime(x))
df['Year'] = df.Date.apply(lambda x: x.year)
df['Period'] = df.Date.apply(lambda x: (x.year // 10) * 10)

In [4]:
df.Route.fillna('', inplace=True)
df.Location.fillna('', inplace=True)

In [5]:
def get_country(location):
    parts = location.split(', ')
    if len(parts) == 2:
        return parts[1]
    else:
        return 'Other'

In [6]:
df['Country'] = df.Location.apply(get_country)

In [7]:
def plot_fatalities(route_point = 'All', route_point_type = 'from', period = [1908, 2009]):
    
    if route_point != 'All':
        if route_point_type == 'from':
            df_part = df[df.Route.str.contains(route_point + ' -')]
        elif route_point_type == 'to':
            df_part = df[df.Route.str.contains('- ' + route_point)]
    else:
        df_part = df
        
    df_part = df_part[df_part.Year >= period[0]]
    df_part = df_part[df_part.Year <= period[1]]
    
    if len(df_part) == 0:
        return None
        
    df_to_plot = df_part.pivot_table(
        index = 'Country',
        values = 'Fatalities',
        aggfunc = np.sum
    )

    df_to_plot['Rank'] = df_to_plot.Fatalities.rank(ascending=False)
    df_to_plot = df_to_plot.reset_index()
    df_to_plot['Group'] = df_to_plot.apply(lambda row: row['Country'] if row['Rank'] <= 10 else 'Other', axis=1)

    df_to_plot = df_to_plot.pivot_table(
        index = 'Group',
        values = 'Fatalities',
        aggfunc = np.sum
    )

    data = []
    trace = go.Pie(
        labels=df_to_plot.index, values=df_to_plot.Fatalities
    )
    data.append(trace)

    layout = go.Layout(
        height=600,
        width=900,
        title='Airplane crashes fatalities by country, {}-{}, {}: {}'.format(
            str(period[0]), str(period[1]), route_point_type, route_point
        ),
        yaxis = dict(title = 'Fatalities'),
        xaxis = dict(title = 'Year'),
        barmode = 'stack'
    )

    fig = go.Figure(data=data, layout=layout)

    iplot(fig, filename='airplane_crashes', show_link=False)

In [8]:
period_widget = widgets.IntRangeSlider(
    value=[1908, 2009],
    min=1908,
    max=2009,
    step=1,
    description='Period, years:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

route_point_widget = widgets.Dropdown(
    options=['All', 'Helsinki', 'Curacao', 'Amsterdam', 'London', 'New York', 'Paris', 'Berlin', 'Moscow'],
    value='All',
    description='Route point, city:',
    disabled=False,
    style={'description_width': 'initial'}
)

route_point_type_widget = widgets.RadioButtons(
    options=['from', 'to'],
    description='Route point type:',
    disabled=False,
    button_style='',
    style={'description_width': 'initial'}
)

box_layout = widgets.Layout(justify_content='space-between')
ui = widgets.HBox([period_widget, route_point_widget, route_point_type_widget], layout=box_layout)

out = widgets.interactive_output(
    plot_fatalities,
    {'period': period_widget, 'route_point': route_point_widget, 'route_point_type': route_point_type_widget}
)

display(ui, out)

Output()